In [ ]:
import math
from nsepy import get_history
import pandas as pd
import requests
import sys
from tvDatafeed import TvDatafeed, Interval
tv = TvDatafeed()
import talib

chat_id_group = "-1002245236166" #Group where Telegram will receive updates, had to add that BOT in that group as admin
TOKEN = "7419116752:AAFTGAcg9uuvl_j7andYX_r_kJfmgsxIKCo"
#Suppressing Console Output
original_stdout = sys.stdout
sys.stdout = None
sys.stdout = original_stdout
df = pd.read_csv("ind_nifty100list.csv")
stocks = df['Symbol'].unique().tolist() #Sending Output string
stocks[12] = 'BAJAJ_AUTO'
stocks[62] = 'M_M'
# INTERVAL_1_MINUTE = "1m", INTERVAL_5_MINUTES = "5m", INTERVAL_15_MINUTES = "15m", INTERVAL_30_MINUTES = "30m", INTERVAL_1_HOUR = "1h", INTERVAL_2_HOURS = "2h", INTERVAL_4_HOURS = "4h", INTERVAL_1_DAY = "1d", INTERVAL_1_WEEK = "1W", INTERVAL_1_MONTH = "1M"
values_list = []
def sendTeleAlert(message, chat_id):
 url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
 response = requests.get(url)

def posSz(entry_price, stoploss, target):
 try:
     posSz = math.floor(abs(float(200 / (entry_price - stoploss))))
    #  print(f'Position sizing for Max Loss 200 is {posSz} \n')
     values_list.append(posSz)  # 0
 except ZeroDivisionError:
     print('Position sizing not found as Entry Price equals Stoploss')
def riskreward(entry_price, stoploss, target):
  try:
    num = float(abs(entry_price - target) / abs(entry_price - stoploss))
    string = f'1:{num}'
    # print(f' The Risk to Reward ratio is {string} \n')
    values_list.append(num) #1
  except ZeroDivisionError:
    print('Risk Reward not found as Entry Price equals Stoploss')
def stocks_calc(entry_price, stoploss, target):
 total_caps = [50000, 100000] #,60000]
 for i in range(len(total_caps)):
  num_stocks = math.floor(float(total_caps[i]/ entry_price))
  total_capital_used = float(entry_price * num_stocks)
  targetgains = float(abs(entry_price - target) * num_stocks)
  maxloss = float(abs(entry_price - stoploss) * num_stocks)
  values_list.append(num_stocks) #2,6,10
  values_list.append(total_capital_used) #3,7,11
  values_list.append(targetgains) #4,8,12
  values_list.append(maxloss) #5,9,13

count = 0
for n in range(len(stocks)):  
 try:
  stock_data = tv.get_hist(stocks[n], exchange='NSE', interval=Interval.in_15_minute, n_bars=25)
  bbands = talib.BBANDS(stock_data['close'], timeperiod=20, nbdevup=2.0,nbdevdn=2.0,matype=0 )
  for i in range(45):
   if stock_data.iloc[-i]['low'] > bbands[0].iloc[-i]: #Sell 
    # print('SELLLLLLL',stock_data.index[-i+1],stocks[n])
    {posSz(stock_data.iloc[-i]['low'], stock_data.iloc[-i]['high'], bbands[1].iloc[-i])} #Low=stock_data.iloc[-i]['low'], BBHIGH=bbands[0].iloc[-i]
    {riskreward(stock_data.iloc[-i]['low'], stock_data.iloc[-i]['high'], bbands[1].iloc[-i])}
    if values_list[1] < 2: #Segregating Risk Reward Ratio > 1:2
     continue
    {stocks_calc(stock_data.iloc[-i]['low'], stock_data.iloc[-i]['high'], bbands[1].iloc[-i])}

    message = f"SELL CASE: {stocks[n]}, Time: {stock_data.index[-i+1]},\n\n Entry Price: {stock_data.iloc[-i]['low']}\n Stoploss: {stock_data.iloc[-i]['high']}\n Target: {bbands[1].iloc[-i]}\n Risk-Reward is 1:{values_list[1]} \n\n Position Sizing is {values_list[0]} stocks for Max Loss of 200 and corresponding Total Capital is {stock_data.iloc[-i]['low']*values_list[0]} with Target Gains of {values_list[0]*(stock_data.iloc[-i]['low']-bbands[1].iloc[-i])} \n\n Total Capital Used is {values_list[3]} \n Number of stocks is {values_list[2]} \n Target Gains = {values_list[4]} \n Max Loss is = {values_list[5]} \n\n Total Capital Used is {values_list[7]} \n Number of stocks is {values_list[6]} \n Target Gains = {values_list[8]} \n Max Loss is = {values_list[9]}"
    print(message)
    sendTeleAlert(message, chat_id_group)
    count += 1
#\n\n Total Capital Used is {values_list[11]} \n Number of stocks is {values_list[10]} \n Target Gains = {values_list[12]} \n Max Loss is = {values_list[13]} \n\n
   elif stock_data.iloc[-i]['high'] < bbands[2].iloc[-i]: #Buy  
    # print(f'BUYYYYYY',stock_data.index[-i+1],stocks[n]) #High=stock_data.iloc[-i]['high'], BBLOW=bbands[2].iloc[-i]
    {posSz(stock_data.iloc[-i]['high'], stock_data.iloc[-i]['low'], bbands[1].iloc[-i])}
    {riskreward(stock_data.iloc[-i]['high'], stock_data.iloc[-i]['low'], bbands[1].iloc[-i])}
    if values_list[1] < 2: #Segregating Risk Reward Ratio > 1:2
     continue
    {stocks_calc(stock_data.iloc[-i]['high'], stock_data.iloc[-i]['low'], bbands[1].iloc[-i])}

    message = f"BUY CASE: {stocks[n]}, Time: {stock_data.index[-i+1]},\n\n Entry Price: {stock_data.iloc[-i]['high']}\n Stoploss: {stock_data.iloc[-i]['low']}\n Target: {bbands[1].iloc[-i]}\n Risk-Reward is 1:{values_list[1]} \n\n Position Sizing is {values_list[0]} stocks for Max Loss of 200 and corresponding Total Capital is {stock_data.iloc[-i]['high']*values_list[0]} with Target Gains of {values_list[0]*(bbands[1].iloc[-i])-stock_data.iloc[-i]['high']} \n\n Total Capital Used is {values_list[3]} \n Number of stocks is {values_list[2]} \n Target Gains = {values_list[4]} \n Max Loss is = {values_list[5]} \n\n Total Capital Used is {values_list[7]} \n Number of stocks is {values_list[6]} \n Target Gains = {values_list[8]} \n Max Loss is = {values_list[9]}\n\n"
#\n\n Total Capital Used is {values_list[11]} \n Number of stocks is {values_list[10]} \n Target Gains = {values_list[12]} \n Max Loss is = {values_list[13]} 
    print(message)
    sendTeleAlert(message, chat_id_group)
    count += 1
   else:
    # print(f'Kuch nhi mila', stock_data.index[-i+1], stocks[n])
    continue
 except ConnectionError:
  print(f"Connection Error processing {stocks[n]}")
 except Exception as e:
  print(f"Exception Error processing {stocks[n]}: {e}")
 values_list.clear()

print(f'Total results are {count}')